In [ ]:
import ipytone
import ipywidgets as widgets

#### Synths, effects and audio routing

In [ ]:
channel1 = ipytone.Channel(pan=-0.5).to_destination()
filtr = ipytone.Filter(frequency=100)
synth = ipytone.MonoSynth(volume=-7).chain(filtr, channel1)

channel2 = ipytone.Channel(pan=0.2, channel_count=2).to_destination()
delay = ipytone.PingPongDelay(delay_time="16n", feedback=0.2)
perc_synth = ipytone.MembraneSynth(volume=-10).chain(delay, channel2)

In [ ]:
lfo = ipytone.LFO(frequency="4m", min=100, max=10_000)
lfo.connect(filtr.frequency).start()

lfo2 = ipytone.LFO(frequency="8n", min=-200, max=200, type="sawtooth")
lfo2.connect(perc_synth.detune).start()

perc_synth.pitch_decay = 0.02
delay.wet.value = 0.1

#### Music loop

In [ ]:
def clb(time, note):
    synth.trigger_attack_release(note, 0.2, time=time)
    perc_synth.trigger_attack_release(note, 0.05, time=time)

sequence = ipytone.Sequence(
    callback=clb,
    events=[None, "D1", "E1", "F2", "F#2", "G2", "G#2", "A2"],
    subdivision="16n",
)

#### Play it!

In [ ]:
ipytone.transport.start()
sequence.start()

Pause right now if you forgot to adjust the volume of the speakers!!

In [ ]:
ipytone.transport.pause()

Ok now? Let's resume and have some fun...

In [ ]:
ipytone.transport.start()

In [ ]:
sequence.events = [None, "D1", "E1", "F3", "F#3", "G3", "G#3", "A4"]

In [ ]:
sequence.events = [None, "D1", "E1", None, "F#4", None, "A5", "A4"]

In [ ]:
synth.portamento = 0.15

In [ ]:
sequence.events = [None, "D1", "E1", "F2", "F#2", "G2", "G#2", "A2"]

#### Mixer (using ipywidgets)

In [ ]:
def create_channel(node):
    pan = widgets.FloatSlider(
        value=node.pan.value, min=-1, max=1,
        layout=widgets.Layout(width="150px")
    )
    fader = widgets.FloatSlider(
        value=0, min=-30, max=4, orientation="vertical"
    )
    vu = widgets.FloatProgress(
        min=0, max=0.3, orientation="vertical"
    )
    solo = widgets.ToggleButton(value=False, description="Solo")
    mute = widgets.ToggleButton(value=False, description="Mute")
    
    widgets.jslink((pan, "value"), (node.pan, "value"))
    widgets.jslink((fader, "value"), (node.volume, "value"))
    
    meter = ipytone.Meter(normal_range=True)
    node.connect(meter)
    meter.schedule_jsdlink((vu, "value"), transport=True)
    
    def node_solo(change):
        node.solo = change['new']
        
    def node_mute(change):
        node.mute = change["new"]
        
    solo.observe(node_solo, names='value')
    mute.observe(node_mute, names='value')
    
    return widgets.VBox([pan, solo, mute, widgets.HBox([fader, vu])])


widgets.HBox([create_channel(channel1), create_channel(channel2)])

#### Enough!

In [ ]:
sequence.stop()

In [ ]:
ipytone.transport.stop()

#### Dispose (free audio ressource)

In [ ]:
synth.dispose()
filtr.dispose()
lfo.dispose()
channel1.dispose()
perc_synth.dispose()
delay.dispose()
channel2.dispose()

sequence.dispose()